In [1]:
from model import LSTMAutoEncoder,SequenceDataset
import pandas as pd
import numpy as np
import os
import glob
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import yaml
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy

In [2]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
train_file_list=glob.glob(config['data_root']+'train/*.csv')
test_file_list=glob.glob(config['data_root']+'test/*.csv')
features=config['features']
traindataset=SequenceDataset(train_file_list,features,mode='train',slide_window=config['slide_windw'], seq_len=config['seq_len'])
traindataloader=DataLoader(traindataset, batch_size=config['batch_size'], shuffle=False, num_workers=0)

testdataset=SequenceDataset(test_file_list,features,mode='test',slide_window=config['slide_windw'], seq_len=config['seq_len'])
testdataloader=DataLoader(testdataset, batch_size=config['batch_size'], shuffle=False, num_workers=0)


In [ ]:
for x_batch, y_batch in traindataloader:
    print(x_batch.shape)  # (batch_size, seq_len, num_features)
    print(y_batch)  # (class)

In [ ]:
def train(config,traindataloader):
    epochs = config['epochs']
    lr = config['learning_rate']
    
    model=LSTMAutoEncoder(input_dim=len(config['features']), hidden_dim=64, num_layers=1)
    optimizer = getattr(optim, config['optimizer'])(model.parameters(), lr=lr)
    train_criterion = getattr(nn, config['train_loss'])()

    for epoch in range(epochs):
        total_loss = 0
        for data in traindataloader:
            optimizer.zero_grad()
            recon = model(data[0])
            train_loss = train_criterion(recon, data[0])
            train_loss.backward()
            optimizer.step()
            total_loss += train_loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(traindataloader):.4f}")

    return model

def test(model_weight,thres,testdataloader):

    model=LSTMAutoEncoder(input_dim=len(config['features']), hidden_dim=64, num_layers=1)
    model.load_state_dict(torch.load(model_weight))
    model.eval()
    with torch.no_grad():
        Y_PRED=[]
        for data in testdataloader:
            recon = model(data[0])
            errors = torch.mean((data[0] - recon)**2, dim=(1,2))
            y_pred = torch.where(errors >= thres, 0, 1)
            Y_PRED.extend(y_pred.cpu().numpy())
        print(Y_PRED)
            # f1_score = BinaryF1Score(y_pred, data[1]) 
            # acc_score = BinaryAccuracy(y_pred, data[1]) 

            # print(f"F1: {f1_score:.4f}, Acc: {acc_score:.4f}")
    

Epoch 1/1000, Loss: 1128249.4260
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

KeyboardInterrupt: 

In [14]:
data=torch.tensor([0.2,0.5,0.6,0.1])
result = torch.where(data >= 0.5, 0, 1)
result

tensor([1, 0, 0, 1])